In [1]:
import pandas as pd
import os
from os.path import isfile, join
import numpy as np
from LabUtils.Scripts.shell_commands_execute import ShellCommandsExecute
def summarize_gwas(numGwases, onlythesecols=None, forHist=False, threshold=False, use_clumped=True,
                   containing_dir="/net/mraid20/export/jasmine/zach/height_gwas/all_gwas/gwas_results/",
                   clump_dir="/net/mraid20/export/jasmine/zach/height_gwas/all_gwas/gwas_results_clumped/",
                   singleBatch=True,
                   onlyRank = False):
    #correct for all the GWASES that we did, not just the ones from each loader.
    if use_clumped:
        containing_dir = clump_dir
        sep = "\s+|\t+|\s+\t+|\t+\s+"
    else:
        sep = "\t"
    if use_clumped:
        all_gwases = [f for f in os.listdir(containing_dir) if
                      isfile(join(containing_dir, f)) and f.endswith(".clumped")]
    else:
        all_gwases = [f for f in os.listdir(containing_dir) if isfile(join(containing_dir, f))]
    if forHist:
        numGwases = 1
        threshold = True
    if onlythesecols is not None:
        ##intersect the columns from the loader with the gwases we actually have
        if singleBatch and not use_clumped:
            all_gwases = list(set(all_gwases).intersection(
                set(list(map(lambda res_name: "batch0." + res_name + ".glm.linear", onlythesecols)))))
        elif not singleBatch and not use_clumped:
            all_gwases = list(
                set(all_gwases).intersection(set(list(map(lambda res_name: res_name + ".glm.linear", onlythesecols)))))
        else:
            all_gwases = list(set(all_gwases).intersection(
                set(list(map(lambda res_name: "batch0." + res_name + ".clumped", onlythesecols)))))
    hits = {}
    rank = dict(
        zip([long_filename.split(".glm.linear")[0] for long_filename in all_gwases], [0 for gwas in all_gwases]))
    i = 1
    for long_filename in all_gwases:
        if (".clumped" in long_filename and use_clumped) or (not use_clumped and ".glm.linear" in long_filename):
            gwas = pd.read_csv(containing_dir + long_filename, sep=sep)
            if len(gwas.P) != 0:  # ignore gwases with no significant associations
                gwas.columns = ["CHROM", *gwas.columns[
                    range(1, len(gwas.columns))]]  ##fix the hashtag in the name of the chromosome column
                if not onlyRank:
                    if not threshold:
                        hits[long_filename.split(".glm.linear")[0]] = gwas
                    else:
                        hits[long_filename.split(".glm.linear")[0]] = gwas.loc[gwas.P < 0.05, :]
                rank[long_filename.split(".glm.linear")[0]] = len(gwas.loc[gwas.P < 0.05, :])
            print(str(i) + "/" + str(len(all_gwases)))
        i += 1
    if not onlyRank:
        hits = pd.Series(hits)
    else:
        hits = None
    rank = pd.Series(rank).sort_values(ascending=False)
    return hits, rank

In [2]:
goodpath = "/net/mraid20/export/genie/LabData/Analyses/10K_Trajectories/biological_age/Xs/"
files = os.listdir(goodpath)
files_women = list(filter(lambda x: x.endswith("female_equal.csv"), files))
corresponding_phenos = dict(zip(files_women, list(map(lambda x: list(pd.read_csv(goodpath + x).columns[1:]), files_women))))

In [3]:
hits_dict, rank_dict = {}, {}

In [4]:
for file, phenos in corresponding_phenos.items(): ##dont trust the phenotype count, do it later, so set numGwases to one for now and do it later
    hits_dict[file], rank_dict[file] = summarize_gwas(numGwases = 1, use_clumped = False, threshold = True, onlythesecols = phenos)

/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:58: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


1/114
2/114
3/114
4/114
5/114
6/114
7/114
8/114
9/114
10/114
11/114
12/114
13/114
14/114
15/114
16/114
17/114
18/114
19/114
20/114
21/114
22/114
23/114
24/114
25/114
26/114
27/114
28/114
29/114
30/114
31/114
32/114
33/114
34/114
35/114
36/114
37/114
38/114
39/114
40/114
41/114
42/114
43/114
44/114
45/114
46/114
47/114
48/114
49/114
50/114
51/114
52/114
53/114
54/114
55/114
56/114
57/114
58/114
59/114
60/114
61/114
62/114
63/114
64/114
65/114
66/114
67/114
68/114
69/114
70/114
71/114
72/114
73/114
74/114
75/114
76/114
77/114
78/114
79/114
80/114
81/114
82/114
83/114
84/114
85/114
86/114
87/114
88/114
89/114
90/114
91/114
92/114
93/114
94/114
95/114
96/114
97/114
98/114
99/114
100/114
101/114
102/114
103/114
104/114
105/114
106/114
107/114
108/114
109/114
110/114
111/114
112/114
113/114
114/114


/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:58: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


1/4
2/4
3/4
4/4
1/182
2/182
3/182
4/182
5/182
6/182
7/182
8/182
9/182
10/182
11/182
12/182
13/182
14/182
15/182
16/182
17/182
18/182
19/182
20/182
21/182
22/182
23/182
24/182
25/182
26/182
27/182
28/182
29/182
30/182
31/182
32/182
33/182
34/182
35/182
36/182
37/182
38/182
39/182
40/182
41/182
42/182
43/182
44/182
45/182
46/182
47/182
48/182
49/182
50/182
51/182
52/182
53/182
54/182
55/182
56/182
57/182
58/182
59/182
60/182
61/182
62/182
63/182
64/182
65/182
66/182
67/182
68/182
69/182
70/182
71/182
72/182
73/182
74/182
75/182
76/182
77/182
78/182
79/182
80/182
81/182
82/182
83/182
84/182
85/182
86/182
87/182
88/182
89/182
90/182
91/182
92/182
93/182
94/182
95/182
96/182
97/182
98/182
99/182
100/182
101/182
102/182
103/182
104/182
105/182
106/182
107/182
108/182
109/182
110/182
111/182
112/182
113/182
114/182
115/182
116/182
117/182
118/182
119/182
120/182
121/182
122/182
123/182
124/182
125/182
126/182
127/182
128/182
129/182
130/182
131/182
132/182
133/182
134/182
135/182
136/182
137/

/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:58: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


1/64
2/64
3/64
4/64
5/64
6/64
7/64
8/64
9/64
10/64
11/64
12/64
13/64
14/64
15/64
16/64
17/64
18/64
19/64
20/64
21/64
22/64
23/64
24/64
25/64
26/64
27/64
28/64
29/64
30/64
31/64
32/64
33/64
34/64
35/64
36/64
37/64
38/64
39/64
40/64
41/64
42/64
43/64
44/64
45/64
46/64
47/64
48/64
49/64
50/64
51/64
52/64
53/64
54/64
55/64
56/64
57/64
58/64
59/64
60/64
61/64
62/64
63/64
64/64


/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:58: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


1/45
2/45
3/45
4/45
5/45
6/45
7/45
8/45
9/45
10/45
11/45
12/45
13/45
14/45
15/45
16/45
17/45
18/45
19/45
20/45
21/45
22/45
23/45
24/45
25/45
26/45
27/45
28/45
29/45
30/45
31/45
32/45
33/45
34/45
35/45
36/45
37/45
38/45
39/45
40/45
41/45
42/45
43/45
44/45
45/45
1/4
2/4
3/4
4/4


In [66]:
for file, phenos in corresponding_phenos.items():
    snps_temp = []
    gwases = hits_dict[file] ##all the gwases from phenos in that file thresholded to standard significance live here, in itself a dictionary
    numTested = 1# len(gwases)
    for gwas in gwases:
        sig_gwas = gwas.loc[gwas.P < 0.05,:]
        snps_temp += list(sig_gwas.ID)
    snps_temp = list(set(snps_temp))
    sig_snps_series = pd.Series(snps_temp)
    sig_snps_series.to_csv("/net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/" + file, header = False, index = False, sep = "\n")

/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/wisdom/python3/lib/python3.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [10]:
def do_subset(file_name, base_dir = "/net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/"):
    file_name = base_dir + "snplists/" + file_name
    out_loc = base_dir + "bins/"
    pheno_name = file_name.split(".csv")[0]
    subset_cmd = "/net/mraid20/export/genie/Bin/plink/plink --bfile /net/mraid20/export/jasmine/zach/height_gwas/all_gwas/gwas_extra_qc/allsamples_qc_final_pass --extract " + file_name + " --noweb --allow-no-sex --make-bed --out " + pheno_name
    return subset_cmd

In [72]:
def do_het(file_name, out_loc = "/net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/"):
    het_cmd = "/net/mraid20/export/genie/Bin/plink2/plink --bfile " + out_loc + file_name + " --het --keep /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/rel.rel.id --noweb --allow-no-sex --out " + file_name
    return het_cmd

In [60]:
def do_king(file_name, out_loc = "/net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/"):
    king_cmd = "/net/mraid20/export/genie/Bin/plink2/plink --bfile /net/mraid20/export/jasmine/zach/height_gwas/all_gwas/gwas_extra_qc/allsamples_qc_final_pass "+ "--rel-cutoff --noweb --allow-no-sex --out /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/rel"
    return king_cmd

In [70]:
##plink first step
##make the subset commands, to paste all at once into the shell
for file_name, phenos, in corresponding_phenos.items():
    if rank_dict[file_name].sum() > 0:
        print(do_subset(file_name) + " &")

/net/mraid20/export/genie/Bin/plink/plink --bfile /net/mraid20/export/jasmine/zach/height_gwas/all_gwas/gwas_extra_qc/allsamples_qc_final_pass --extract /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/body_composition_female_equal.csv --noweb --allow-no-sex --make-bed --out /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/body_composition_female_equal &
/net/mraid20/export/genie/Bin/plink/plink --bfile /net/mraid20/export/jasmine/zach/height_gwas/all_gwas/gwas_extra_qc/allsamples_qc_final_pass --extract /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/frailty_female_equal.csv --noweb --allow-no-sex --make-bed --out /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/frailty_female_equal &
/net/mraid20/export/genie/Bin/plink/plink --bfile /net/mraid20/export/jasmine/zach/height_gwas/all_gwas/gwas_extra_qc/allsamples_qc_final_pass --extract /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/bone_density_female_eq

In [62]:
##plink second step: removing first degree relations and duplicate samples (from both v1 and v2 gencove APIs):
for file_name, phenos, in corresponding_phenos.items():
    if rank_dict[file_name].sum() > 0:
        print(do_king(file_name.split(".csv")[0]))
        break ##we only need do this once, as the sample ids are shared across all binaries

/net/mraid20/export/genie/Bin/plink2/plink --bfile /net/mraid20/export/jasmine/zach/height_gwas/all_gwas/gwas_extra_qc/allsamples_qc_final_pass --rel-cutoff --noweb --allow-no-sex --out /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/


In [73]:
##plink third step
##make the het commands, to paste all at once into the shell
for file_name, phenos, in corresponding_phenos.items():
    if rank_dict[file_name].sum() > 0:
        print(do_het(file_name.split(".csv")[0]) + " &")

/net/mraid20/export/genie/Bin/plink2/plink --bfile /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/body_composition_female_equal --het --keep /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/rel.rel.id --noweb --allow-no-sex --out body_composition_female_equal &
/net/mraid20/export/genie/Bin/plink2/plink --bfile /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/frailty_female_equal --het --keep /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/rel.rel.id --noweb --allow-no-sex --out frailty_female_equal &
/net/mraid20/export/genie/Bin/plink2/plink --bfile /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/bone_density_female_equal --het --keep /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/rel.rel.id --noweb --allow-no-sex --out bone_density_female_equal &
/net/mraid20/export/genie/Bin/plink2/plink --bfile /net/mraid20/ifs/wisdom/segal_lab/jasmine/zach/saar_het/snplists/sleep_female_equal --h